https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html

MNIST

# Import modules

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Define the network

In [6]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        self.conv1 = nn.Conv2d(1, 6, 5)  # 32x32x3 -> 14x14x6
        self.conv2 = nn.Conv2d(6, 16, 5)  # 14x14x6 -> 5x5x16
        # affine
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions expect the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [7]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


learnable parameters

In [9]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's weight

10
torch.Size([6, 1, 5, 5])


↑で構築した LeNet の入力サイズは 32x32 なので、MNIST データセットをリサイズする必要がある。

In [12]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0352,  0.0644, -0.0352, -0.0129, -0.1005,  0.1090, -0.0078, -0.0096,
          0.0651, -0.1443]], grad_fn=<ThAddmmBackward>)


In [13]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [15]:
out

tensor([[ 0.0352,  0.0644, -0.0352, -0.0129, -0.1005,  0.1090, -0.0078, -0.0096,
          0.0651, -0.1443]], grad_fn=<ThAddmmBackward>)

# Loss function

In [19]:
output = net(input)
target = torch.randn(10)  # a dummy target
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.9490, grad_fn=<MseLossBackward>)


In [20]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

# Backprop

In [21]:
net.zero_grad()

print("conv1.bias.grad before backward")
print(net.conv1.bias.grad)

loss.backward()

print("conv1.bias.grad after backward")
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0119,  0.0039,  0.0201,  0.0067,  0.0019, -0.0025])


# Update the weights

SGD

weight = weight - learning_rate * gradient

In [23]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [26]:
optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()  # update